# <center>ANBNN: AN Codes for Ternary-Coded Binarisied Neural Network</center>
<center>TestLab, NCUE, Taiwan</center>
<center>2020/12/15</center>

* References
* http://www.ecs.umass.edu/ece/koren/FaultTolerantSystems/simulator/Arith/arithcode.html
* https://en.wikipedia.org/wiki/AN_codes

In [ ]:
from TCB import *
from math import *
a = 15*32+7
t = Bin2TCB(a)
b = TCB2Bin(t)
s = '{0:b}'.format(b)
a, b, t, Weight(a), SWeight(t), s

In [ ]:
from RNS import *
a = 120
b = 84
gcd(a,b), lcm(a, b)

In [ ]:
import math
def isPrime(n):
    p=1
    i=2
    while (i * i <= n):
        if math.gcd(i, n)!=1:
            p=0
        i += 1
    return p

def primes(N):
    p = []
    for i in range(2, N+1):
        if isPrime(i):
            p.append(i)
    return p
print(primes(1000))

In [ ]:
### Sieve of Eratosthenes (愛氏篩法、質數篩法):
def sievePrimes(n): 
    prime = [True for i in range(n + 1)] 
    prime[0]= False
    prime[1]= False
    p = 2
    while (p * p <= n): 
        if (prime[p] == True): 
            for i in range(p * 2, n + 1, p): 
                prime[i] = False
        p += 1
    for p in range(n + 1): 
        if prime[p]: 
            print(p, end=' ')
sievePrimes(1000)

In [ ]:
#(2^n-1)為質數者
def primeMBN(n):
    p=[]
    for i in range(2, n+1):
        N = 2**i -1
        if isPrime(N):
            p.append([i, N])
    return p
print(primeMBN(32))

In [ ]:
#質因數分解遞迴演算法
def defactor(p, n):
    if(p>math.sqrt(n)):
        return [n]
    elif math.gcd(p, n)==p:
        return [p] + defactor(p, n//p)
    else:
        return defactor(p+1, n)
defactor(2, 2873150280), defactor(2, 12345678)

In [ ]:
# 質因數分解
def factorize(n):
    factor = []
    p = 2
    m = 0
    q = n
    while(q>=p):
        if math.gcd(p, q)==p:
            m += 1
            q //= p
        else:
            if m>0:
                factor.append([p, m])
            m = 0
            p += 1
    if m>0:
        factor.append([p, m])
    return factor
factorize(2873150280)

In [ ]:
# integer division
print(12 // 3, 11 // 3, 11/3)

In [ ]:
#Mandelbaum-Barrows Codes與AW distance d
def MBCodes(n):
    for i in range(2, n+1):
        N = 2**i - 1
        f = factorize(N)
        B = f[-1][0]
        r = 2
        d = (i/(B-1))*(ceil(2*B/(2+1))-ceil(B/(2+1)))
        print(i, N, f, d)
MBCodes(30)

In [ ]:
# bit-format
def bF(n):
    return '{0:b}'.format(n)
print(bF(37))

In [5]:
#如果q為質數，則{0, 1, 2, .., q-1}構成葛瑞場(Galois Field), GF(q)
# 加法群：0是加法反身元素，q-i是i的反元素; 除了0之外，係由「+1」產生(Generator)
# 乘法群：1是乘法反身元素，擬比照加法群來產生乘法群，可以當Generator的就稱之為primitive element(原生元素)
# 最小的非1原生元素r排在前，依次產生{0 ,r^0, r^1, r^2, ..., r^(q-2)}
# 費馬-尤拉定理: r^(q-1) = r^0 = 1 (mod q)
# 假設 q-1可以分解成 2x3x..., 則指數為2, 3, ...的倍數的不是primitive elements, 剩下的就是primitive elements
# 2為primitive element的q比較適合當作(q/2)位元AN Codes的A
q = 13
def primitiveElementsOfGF(q):
    PE = []
    if isPrime(q):
        for r in range(2, q):
            found = 1
            for k in range(1, q-1):
                if ((r ** k) % q == 1):
                    found = 0
                    break
            if found:
                break
        PE = [0, 1, r]
        for i in range(2, q-1):
            PE.append((PE[-1]*r)%q)
    return PE
print('GF(%d) = ' % (q), primitiveElementsOfGF(q))

GF(13) =  [0, 1, 2, 4, 8, 3, 6, 12, 11, 9, 5, 10, 7]


In [ ]:
primitiveElementsOfGF(12)

In [ ]:
p=13
s = "file" + str(1933)
s

In [7]:
#### AN-Code Decoder
# 2025/2/10
# Using isPrime(p), primitiveElementsOfGF(p)
import math
def isPrime(n):
    p=1
    i=2
    while (i * i <= n):
        if math.gcd(i, n)!=1:
            p=0
        i += 1
    return p

def primitiveElementsOfGF(q):
    PE = []
    if isPrime(q):
        for r in range(2, q):
            found = 1
            for k in range(1, q-1):
                if ((r ** k) % q == 1):
                    found = 0
                    break
            if found:
                break
        PE = [0, 1, r]
        for i in range(2, q-1):
            PE.append((PE[-1]*r)%q)
    return PE

#### AN-Code Decoder
# 2025/2/10
# Using isPrime(p), primitiveElementsOfGF(p)
#-------------------------------------
p = 13
Bp = 4
Bw = 6 
Bn = Bw - Bp

#-------------------------------------
Remainders = primitiveElementsOfGF(p)
#-------------------------------------
fn = 'AN_Code_Decoder' + str(p) + '.v'
f = open(fn, 'w')

f.write('// Product (AN) Code Decoder\n')
f.write('// ' + fn + '\n')
f.write('// Received codeword W = AN + e, e is single arithmetic weight error (AWE), +2^i or -2^i.\n')
f.write('module ProductDecoder(W, N);\n')
f.write('input \t[%d:0]\tW;\n' % (Bw-1))
f.write('output\t[%d:0]\tN;\n' % (Bn-1))
f.write('parameter A = %d;\n\n' % (p))
f.write('wire \t[%d:0]\tQ;\n' % (Bn-1))
f.write('wire \t[%d:0]\tR;\n' % (Bp-1))
f.write('assign Q = W / A;\n')
f.write('assign R = W - (A * Q);\n\n')

f.write('reg \t[%d:0]\tDelta;\n' % (Bw-1))
f.write('always@(*) begin\n')
f.write('\tcase(R)\n')
for i in range(Bw):
    f.write('\t\t%d: Delta = %d;\n' %(Remainders[i+1], 1<<i))
for i in range(Bw, 2*Bw):
    f.write('\t\t%d: Delta = -%d;\n' %(Remainders[i+1], 1<<(i-Bw)))
f.write('\tendcase\n')
f.write('end\n\n')
f.write('assign N = (W - Delta) / A;\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file AN_Code_Decoder13.v is generated.



In [ ]:
for q in range(3, 132):
    if isPrime(q):
        print('GF(%d) = ' % (q), primitiveElementsOfGF(q))

In [ ]:
# 列出可以當AN Codes (AN有n位元)的最小A及其Weight; 最小Weight的A, 及其Weight
from TCB import *
#k = 3
k = 10
for n in range(1, 129):
    Alight = 0
    for A in range(2*n+1, k*n+1, 2):
        r = []
        for i in range(n):
            e = 2**i
            r.append(e%A)
            r.append((-e)%A)
        identity=1
        for i in range(len(r)-1):
            for j in range(i+1, len(r)):
                if r[i]==r[j]:
                    identity=0
        if identity:
            if Alight == 0:
                Amin, Alight = A, A
            elif Weight(A)<Weight(Alight):
                Alight = A
    #print(n, A, r)
    print(n, Amin, Weight(Amin), Alight, Weight(Alight))

In [ ]:
### 費馬-歐拉定理Fermat-Euler Theorem
### 歐拉函數Euler Totient Function
def phi(n):
    k=0
    for i in range(n):
        if gcd(i, n)==1:
            k+=1
    return k
phi(5)
### Fermat-Euler Theorem
for a in range(3, 100, 2):
    print("%d:%d" % (a, 2**phi(a) % a), end=' ')

In [ ]:
phi(10)

In [ ]:
### Minum length of Distinct Plus-1-Group and Minus-1-Group of q
def Cosets(q):
    Gp, Gm = [1], [-1 % q]
    while True:
        Np = (Gp[-1] * 2) % q
        Nm = (Gm[-1] * 2) % q
        if (Np in Gp) or (Np in Gm) or (Nm in Gp) or (Nm in Gm) or (Np == Nm):
            break
        else:
            Gp.append(Np)
            Gm.append(Nm)
    return [Gp, Gm]

In [ ]:
for q in range(3, 132):
    n = len(Cosets(q)[0])
    rate = int(200 * n / (q-1))
    print("%d: %d  %d" % (q, n, rate))

In [ ]:
## Natural Numbers: 0
## Composite numbers: 1
## Prime numbers: 2
## Perfect Cosets: 3
## 2-primitive numbers: 4
S=[0, 0, 2]
N=256
for q in range(3, N+1):
    S.append(1)
    if isPrime(q):
        S[-1] += 1
        if len(Cosets(q)[0]) * 2 == q - 1:
            S[-1] += 1
            if primitiveElementsOfGF(q)[2]==2:
                S[-1] += 1
print("%10s|%10s|%10s(%10s(%10s))" % ('special', 'composite', 'prime  ', 'max-cosets', 'primitive'))
for q in range(N+1):
    if S[q]==0:
        print("%6s%d%4s%11s" % ('', q, '|', '|'))
    elif S[q]==1:
        print("%10s%s%6d%5s" % ('', '|', q, '|'))
    elif S[q]==2:
        print("%11s%11s%6d" % ('|', '|', q))
    elif S[q]==3:
        print("%11s%11s%17d" % ('|', '|', q))
    else:
        print("%11s%11s%28d" % ('|', '|', q))        

In [ ]:
## Proving that P1P2N Codes are wrong since the residues are not unique to locate the double errors and single errors
P1 = 23
P2 = 47
n = (P1-1)//2
k = n - math.log2(P1*P2)
R = []
for i in range(n-1):
    for si in range(2):
        for j in range(i+1, n):
            for sj in range(2):
                r = ((-1)**si)*(2**i) + ((-1)**sj)*(2**j)
                r1 = r % P1
                r2 = r % P2
                R.append(r2*100+r1)
R.sort()

repeat = 0
for i in range(len(R)-1):
    if R[i]==R[i+1]:
        repeat += 1
L = len(R)
n, k, (L-repeat)/L, R

In [ ]:
## Proving that P1P2N Codes are wrong since the residues are not unique to locate the double errors and single errors
P1 = 53
P2 = 59
n = (P1-1)//2
k = n - math.log2(P1*P2)
R = []
for Sign in [-1, 1]:
    for e in range(1, 2**n):
        if Weight(e) < 3:
            err = Sign * e
            r1 = err % P1
            r2 = err % P2
            R.append([r2*100+r1, err])
R.sort()

repeat = 0
for i in range(len(R)-1):
    if R[i][0]==R[i+1][0]:
        repeat += 1
        print('(%d, %s=%d, %s=%d)' % (R[i][0], Bin2TCB(R[i][1]), R[i][1], Bin2TCB(R[i+1][1]), R[i+1][1]), end=', ')
L = len(R)
n, k, (L-repeat)/L, R

In [ ]:
Bin2TCB(8160)